Run first the [setup notebook](./00-setup.ipynb)

# Classic Text Retrieval with SQL

In [ ]:
from helpers import bag_of_words, tokenize, eliminate_stopwords
from itertools import groupby
import sqlite3
from datasets import imdb
import ipywidgets as widgets

## Create Inverted Index with SQL

### Create Schema

In [ ]:
# helper function to create dict results for SELECT statements
def dict_from_row(cursor, row):
    return {
        col[0]: row[idx]
        for idx, col in enumerate(cursor.description)
    }

# open in-memory database - we only want to demonstrate the SQL interface
db = sqlite3.connect(":memory:")
db.row_factory = dict_from_row

# create table for inverted index in SQL
db.execute("CREATE TABLE document(id INTEGER PRIMARY KEY AUTOINCREMENT, title TEXT, year INTEGER, runtime INTEGER, rating REAL, genre TEXT, actors TEXT, summary TEXT)")
db.execute("CREATE TABLE vocabulary(term TEXT PRIMARY KEY, df INTEGER, idf REAL)")
db.execute("CREATE TABLE posting(term TEXT, docId INTEGER, tf INTEGER)")
db.execute("CREATE TEMPORARY TABLE query(term TEXT, tf INTEGER)")
db.execute("CREATE INDEX inverted_list ON posting(term)");

### Load IMDB data set

In [ ]:
# loading the imdb data set (1000 movies)
collection = imdb.load()
def doc_format_imdb(doc: dict) -> str:
    trim = lambda s,n: len(s) > n and s[:n] + "\u2026" or s
    title_ex = '{title_short} ({year}, {runtime}m, {rating})'.format(title_short=trim(doc['title'], 30), **doc)
    return '{title_ex:<50} {genre_short:<20} {summary} [{actors}]'.format(title_ex=title_ex, genre_short=trim(doc['genre'], 18), **doc)

for item in collection[:5]:
    print(doc_format_imdb(item))

collection[0]

### Build index in SQL

In [ ]:
def get_vector(text: str) -> set:
    tokens = tokenize(text)
    tokens = eliminate_stopwords(tokens)
    # tokens = reduce_to_stems(tokens)
    return bag_of_words(tokens)

def add_document(doc: dict):
    # add new document and fetch doc id
    db.execute("INSERT INTO document(title, year, runtime, rating, genre, actors, summary) VALUES (:title, :year, :runtime, :rating, :genre, :actors, :summary)", doc)
    doc_id = db.execute("SELECT last_insert_rowid() AS docId").fetchone()['docId']
    # create vector from str-properties
    text = ' '.join([value for key, value in doc.items() if type(value) == str])
    vector = get_vector(text)
    # insert postings into table posting
    db.executemany("INSERT INTO posting(term, docId, tf) VALUES(?,?,?)", [(term, doc_id, tf) for term, tf in vector.items()])
    
def build_index(collection: list[dict]):    
    # remove current entries
    db.execute("DELETE FROM posting")
    db.execute("DELETE FROM vocabulary")
    db.execute("DELETE FROM document")
    # add all documents in collection
    for doc in collection:
        add_document(doc)
    # build inverted index table
    n_docs = db.execute("SELECT count(*) AS count FROM document").fetchone()['count']
    db.execute("INSERT INTO vocabulary(term, df, idf) SELECT term, count(*), ln(1.0 * (? + 1) / (count(*) + 1)) FROM posting GROUP BY term", (n_docs, ));
    # commit changes    
    db.commit()

# build index for movie data set
build_index(collection)

# print number of documents
n_docs = db.execute("SELECT count(*) AS count FROM document").fetchone()['count']
print('{count} documents in collection'.format(count=n_docs))

# print number of terms
nTerms = db.execute("SELECT count(*) AS count FROM vocabulary").fetchone()['count']
print('{count} distinct terms in collection'.format(count=nTerms))

# print number of postings
nPostings = db.execute("SELECT count(*) AS count FROM posting").fetchone()['count']
print('{count} postings'.format(count=nPostings))

### Print document table

In [ ]:
cur = db.execute("SELECT * FROM document")
for doc in cur.fetchmany(size=10):
    print(f"{doc['id']:>2} {doc_format_imdb(doc)}")

### Print terms

In [ ]:
cur = db.execute("SELECT * FROM vocabulary ORDER BY df DESC")
for term in cur.fetchmany(size=20):
    print(term)

### Print postings

In [ ]:
cur = db.execute("SELECT * FROM posting")
for posting in cur.fetchmany(size=20):
    print(posting)

## Queries

### Boolean search: AND with 2 terms

In [ ]:
cur = db.execute("""
    SELECT d.* 
      FROM document d, posting a, posting b 
     WHERE a.term = 'star' AND
           b.term = 'wars' AND
           a.docId = b.docId AND
           a.docId = d.id
""")
for doc in cur.fetchmany(size=10):
    print(doc_format_imdb(doc))

### Boolean search: OR with 2 terms

In [ ]:
cur = db.execute("""
    SELECT d.* 
      FROM document d, posting a 
     WHERE a.term IN ('star', 'wars') AND
           a.docId = d.id
""")
for doc in cur.fetchmany(size=10):
    print(doc_format_imdb(doc))

### Boolean search: AND with arbitrary number of terms

In [ ]:
def search_bool(query: str, k: int):
    query_vector = get_vector(query)
    db.execute("DELETE FROM query")
    db.executemany("INSERT INTO query(term, tf) VALUES(?,?)", [(term, tf) for term, tf in query_vector.items()])
    cur = db.execute("""
        SELECT d.* 
          FROM document d, posting p, query q 
         WHERE p.term = q.term AND
               p.docId = d.id
      GROUP BY p.docId
        HAVING COUNT(p.term) = (SELECT COUNT(*) FROM query)
    """)
    print("\n    r   id   document\n" + '-'*160)
    for rank, doc in enumerate(cur.fetchmany(size=k)):
        print('  {rank:>3d} {id:>4d}   {doc}'.format(rank=rank + 1, id=doc['id'], doc=doc_format_imdb(doc)))
    db.commit()

# options for the dialog
queries = ['star wars', 'drama morgan freeman', 'comedy']

# interactive selection of scenario
widgets.interact(search_bool, 
    query=widgets.Dropdown(options=queries), 
    k=widgets.IntSlider(min=5, max=50, step=5, value=20),
);

### Boolean search: OR with arbitrary number of terms

In [ ]:

def search_bool(query: str, k: int):
    query_vector = get_vector(query)
    db.execute("DELETE FROM query")
    db.executemany("INSERT INTO query(term, tf) VALUES(?,?)", [(term, tf) for term, tf in query_vector.items()])
    cur = db.execute("""
        SELECT d.* 
          FROM document d, posting p, query q 
         WHERE p.term = q.term AND
               p.docId = d.id
      GROUP BY p.docId
    """)
    print("\n    r   id   document\n" + '-'*160)
    for rank, doc in enumerate(cur.fetchmany(size=k)):
        print('  {rank:>3d} {id:>4d}   {doc}'.format(rank=rank + 1, id=doc['id'], doc=doc_format_imdb(doc)))
    db.commit()

# options for the dialog
queries = ['star wars', 'drama morgan freeman', 'comedy']

# interactive selection of scenario
widgets.interact(search_bool, 
    query=widgets.Dropdown(options=queries), 
    k=widgets.IntSlider(min=5, max=50, step=5, value=20),
);

### Vector Space Model with dot-product

In [ ]:
def search_vsm(query: str, k: int):
    query_vector = get_vector(query)
    db.execute("DELETE FROM query")
    db.executemany("INSERT INTO query(term, tf) VALUES(?,?)", [(term, tf) for term, tf in query_vector.items()])
    cur = db.execute("""
        SELECT SUM(p.tf * v.idf * q.tf * v.idf) AS score, d.*
          FROM document d, posting p, query q, vocabulary v
         WHERE p.term = q.term AND
               p.term = v.term AND
               p.docId = d.id
      GROUP BY p.docId
      ORDER BY 1 DESC
    """)
    print("\n    r   id  score  document\n" + '-'*160)
    for rank, doc in enumerate(cur.fetchmany(size=k)):
        print('  {rank:>3d} {id:>4d} ({score:5.1f})  {doc}'.format(rank=rank + 1, id=doc['id'], score=doc['score'], doc=doc_format_imdb(doc)))
    db.commit()

# options for the dialog
queries = ['star wars', 'drama morgan freeman', 'comedy']

# interactive selection of scenario
widgets.interact(search_vsm, 
    query=widgets.Dropdown(options=queries), 
    k=widgets.IntSlider(min=5, max=50, step=5, value=20),
);

### Vector Space Model with dot-product with predicate (year > 1990)

In [ ]:
def search_vsm(query: str, k: int):
    query_vector = get_vector(query)
    db.execute("DELETE FROM query")
    db.executemany("INSERT INTO query(term, tf) VALUES(?,?)", [(term, tf) for term, tf in query_vector.items()])
    cur = db.execute("""
        SELECT SUM(p.tf * v.idf * q.tf * v.idf) AS score, d.*
          FROM document d, posting p, query q, vocabulary v
         WHERE p.term = q.term AND
               p.term = v.term AND
               p.docId = d.id AND
               d.year > 1990
      GROUP BY p.docId
      ORDER BY 1 DESC
    """)
    print("\n    r   id  score  document\n" + '-'*160)
    for rank, doc in enumerate(cur.fetchmany(size=k)):
        print('  {rank:>3d} {id:>4d} ({score:5.1f})  {doc}'.format(rank=rank + 1, id=doc['id'], score=doc['score'], doc=doc_format_imdb(doc)))
    db.commit()

# options for the dialog
queries = ['star wars', 'drama morgan freeman', 'comedy']

# interactive selection of scenario
widgets.interact(search_vsm, 
    query=widgets.Dropdown(options=queries), 
    k=widgets.IntSlider(min=5, max=50, step=5, value=20),
);

---